In [597]:
import xml.etree.ElementTree as et
import dateutil.parser as parser
import pandas as pd
import datetime
from timezone import get_time_by_local


# tree = et.parse('xmls/RS_Via-3.xml')
# root = tree.getroot()


class Query:
    def __init__(self, xml):
        self.xml = xml

    def get_xml_root(self):
        tree = et.parse(self.xml)
        root = tree.getroot()
        return root


# class ParseTrips:
#     def __init__(self, root):
#         self.root = root
#     def get_trips(self):


first_q = Query('xmls/RS_ViaOW.xml')
root = first_q.get_xml_root()

steps_dict = {'OnwardPricedItinerary': 'start',
              'ReturnPricedItinerary': 'finish', 'Pricing': 'price'}

CONFIG_FLIGHT = \
    {
        'Carrier': {
            'dtype': 'text',
            'attr': {
                'id': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Airline_id'
                }
            },
            'descrip': 'Airline'
        },
        'FlightNumber': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Number_of_flight'
        },
        'Source': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'From'
        },
        'Destination': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'To'
        },
        'DepartureTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_from'
        },
        'ArrivalTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_to'
        },
        'Class': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Class'
        },
        'NumberOfStops': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Count_stops'
        },
        'FareBasis': {
            'dtype': 'id',
            'attr': False,
            'descrip': 'Trip_id'
        },
        'WarningText': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Description'
        },
        'TicketType': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Type_ticket'
        }

    }
CONFIG_PRICE = \
    {
        'ServiceCharges': {
            'dtype': 'float',
            'attr': {
                'type': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Person_type'
                },
                'ChargeType': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Rate_type'
                }
            },
            'descrip': 'Charges'
        },
    }


def parse_step(node):
    tag = node.tag
    l_step = list()
    if tag in steps_dict:
        l_step = parse_flights(node, tag)
    return l_step


def get_tag(root, config, attr=None):
    tags = dict()
    for tag in config:
        if attr:
            item = root.attrib
            if len(item) > 1:
                item = {k: item[k] for k in item if k == tag}
        else:
            item = root.find(tag) if root is not None else None
        if config[tag]['attr']:
            tags.update(get_tag(item, config[tag]['attr'], attr=True))
        data = check_dtype(item, config, attr)
        tags.update(data)
    return tags


def check_dtype(item, config, attr):
    d = dict()
    if attr is not None:
        tag = next(iter(item))
        data = item[tag]
    else:
        tag = item.tag
        data = item.text
    if tag in config:
        descrip = config[tag]['descrip']
        dtype = config[tag]['dtype']
        if dtype == 'datetime':
            data = parser.parse(data)
        elif dtype == 'int':
            data = int(data)
        elif dtype == 'float':
            data = float(data)
        elif dtype == 'id':
            data = hash(data.strip())
    d[descrip] = data
    return d


def parse_flights(node, tag):
    trip_flights = list()
    transfer_flight = None
    for cur_flight in range(len(node.findall('.//Flight')) - 1, -1, -1):
        row = get_tag(node.findall('.//Flight')[cur_flight], CONFIG_FLIGHT)
        row['Tag'] = tag
        from_local_t = get_time_by_local(row['From'], row['Time_from'])
        to_local_t = get_time_by_local(row['To'], row['Time_to'])
        if from_local_t['code'] == 200 and to_local_t['code'] == 200:
            row['Time_from_local'] = parser.parse(from_local_t['data'])
            row['Time_to_local'] = parser.parse(to_local_t['data'])
        if cur_flight > 0:
            row['transfer_to'] = transfer_flight
            transfer_flight = row['Number_of_flight']
        else:
            row['transfer_to'] = transfer_flight
        trip_flights.append(row)
    return trip_flights


def parse_trip_price(trip, trip_id):
    trip_prices = list()
    for price in trip.findall('.//Pricing'):
        for single_price in price.findall('ServiceCharges'):
            price_d = dict()
            price_d.update(check_dtype(single_price, CONFIG_PRICE, attr=None))
            price_d['currency'] = price.attrib['currency']
            price_d['Trip_id'] = trip_id
            price_attrib = single_price.attrib
            if price_attrib:
                for k in price_attrib:
                    item = {k: price_attrib[k]}
                    price_d.update(
                        check_dtype(item,
                                    CONFIG_PRICE[single_price.tag]['attr'],
                                    attr=True))
            trip_prices.append(price_d)
    return trip_prices


def get_trips(root):
    if 'Flights' == root.tag:
        return [root]
    flights = list()
    for child in root:
        flights += get_trips(child)
    return flights


t1 = datetime.datetime.now()
trips = get_trips(root)
flights = list()
prices = list()
for trip in trips:
    flight = list()
    for trips_step in list(trip):
        trips_step = parse_step(trips_step)
        if trips_step:
            flight.extend(trips_step)
    if flight:
        flights.extend(flight)
        trip_id = flight[0]['Trip_id']
    price = parse_trip_price(trip, trip_id)
    if price:
        prices.extend(price)
df = pd.DataFrame(data=flights)
df = df.sort_values(
    ['Trip_id', 'Tag', 'transfer_to', 'Time_from_local']).reset_index()
prices = pd.DataFrame(data=prices)
print(datetime.datetime.now() - t1)

# k = 0
# for i, flight in df.iterrows():
#     if i == 0:
#         trip_id = flight['Trip_id']
#         tag = flight['Tag']
#         start_time = flight['Time_from_local']
#         finish_time = flight['Time_to_local']
#     elif trip_id == flight['Trip_id'] and tag == flight['Tag']:
#         pause = flight['Time_from_local'] - finish_time
#     elif trip_id != flight['Trip_id'] or tag != flight['Tag']:
#         all_time = df.loc[i - 1, 'Time_to_local'] - start_time
#         trip_id = flight['Trip_id']
#         tag = flight['Tag']
#         start_time = flight['Time_from_local']
#         finish_time = flight['Time_to_local']
#         print(all_time)
# print(200)


0:00:00.374983


In [580]:
df

,index,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
0,254,Pakistan International Airlines,PK,U,0,None,DXB,204,OnwardPricedItinerary,2018-10-27 13:00:00,2018-10-27 12:00:00,2018-10-27 17:00:00,2018-10-27 15:00:00,LHE,-9164081078935422697,E,346.0
1,253,Thai,TG,Y,0,None,LHE,346,OnwardPricedItinerary,2018-10-28 00:05:00,2018-10-27 22:05:00,2018-10-28 06:30:00,2018-10-28 02:30:00,BKK,-9164081078935422697,E,NaN
2,133,China Eastern Airlines,MU,R,0,None,DXB,756,OnwardPricedItinerary,2018-10-27 20:45:00,2018-10-27 19:45:00,2018-10-28 07:15:00,2018-10-28 02:15:00,KMG,-9000761809758366288,E,741.0
3,132,China Eastern Airlines,MU,R,0,None,KMG,741,OnwardPricedItinerary,2018-10-28 14:00:00,2018-10-28 09:00:00,2018-10-28 14:55:00,2018-10-28 10:55:00,BKK,-9000761809758366288,E,NaN
4,284,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8795564162163048642,E,780.0
5,283,Malaysia Airlines,MH,S,0,None,KUL,780,OnwardPricedItinerary,2018-10-27 18:05:00,2018-10-27 13:05:00,2018-10-27 19:10:00,2018-10-27 15:10:00,BKK,-8795564162163048642,E,NaN
6,330,Qantas,QF,S,0,None,DXB,8354,OnwardPricedItinerary,2018-10-27 03:15:00,2018-10-27 02:15:00,2018-10-27 14:40:00,2018-10-27 09:40:00,SIN,-8596059523334321759,E,414.0
7,329,Thai,TG,Y,0,None,SIN,414,OnwardPricedItinerary,2018-10-27 15:55:00,2018-10-27 10:55:00,2018-10-27 17:15:00,2018-10-27 13:15:00,BKK,-8596059523334321759,E,NaN
8,286,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8544928968329196853,E,796.0
9,285,Malaysia Airlines,MH,S,0,None,KUL,796,OnwardPricedItinerary,2018-10-27 21:55:00,2018-10-27 16:55:00,2018-10-27 23:00:00,2018-10-27 19:00:00,BKK,-8544928968329196853,E,NaN


In [601]:

df = df.sort_values(['Trip_id','Tag','transfer_to','Time_from_local'])

df

,index,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
0,254,Pakistan International Airlines,PK,U,0,None,DXB,204,OnwardPricedItinerary,2018-10-27 13:00:00,2018-10-27 12:00:00,2018-10-27 17:00:00,2018-10-27 15:00:00,LHE,-9164081078935422697,E,346.0
1,253,Thai,TG,Y,0,None,LHE,346,OnwardPricedItinerary,2018-10-28 00:05:00,2018-10-27 22:05:00,2018-10-28 06:30:00,2018-10-28 02:30:00,BKK,-9164081078935422697,E,NaN
2,133,China Eastern Airlines,MU,R,0,None,DXB,756,OnwardPricedItinerary,2018-10-27 20:45:00,2018-10-27 19:45:00,2018-10-28 07:15:00,2018-10-28 02:15:00,KMG,-9000761809758366288,E,741.0
3,132,China Eastern Airlines,MU,R,0,None,KMG,741,OnwardPricedItinerary,2018-10-28 14:00:00,2018-10-28 09:00:00,2018-10-28 14:55:00,2018-10-28 10:55:00,BKK,-9000761809758366288,E,NaN
4,284,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8795564162163048642,E,780.0
5,283,Malaysia Airlines,MH,S,0,None,KUL,780,OnwardPricedItinerary,2018-10-27 18:05:00,2018-10-27 13:05:00,2018-10-27 19:10:00,2018-10-27 15:10:00,BKK,-8795564162163048642,E,NaN
6,330,Qantas,QF,S,0,None,DXB,8354,OnwardPricedItinerary,2018-10-27 03:15:00,2018-10-27 02:15:00,2018-10-27 14:40:00,2018-10-27 09:40:00,SIN,-8596059523334321759,E,414.0
7,329,Thai,TG,Y,0,None,SIN,414,OnwardPricedItinerary,2018-10-27 15:55:00,2018-10-27 10:55:00,2018-10-27 17:15:00,2018-10-27 13:15:00,BKK,-8596059523334321759,E,NaN
8,286,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8544928968329196853,E,796.0
9,285,Malaysia Airlines,MH,S,0,None,KUL,796,OnwardPricedItinerary,2018-10-27 21:55:00,2018-10-27 16:55:00,2018-10-27 23:00:00,2018-10-27 19:00:00,BKK,-8544928968329196853,E,NaN


In [602]:
task_airports = ['DXB','BKK']
trip_struct = dict()
trips = list()
pauses = list()
for i, flight in df.iterrows():
    if i == 0:
        start_f = flight
    elif trip_id != flight['Trip_id'] or tag != flight['Tag']:
        all_time = df.loc[i - 1, 'Time_to_local'] - start_f['Time_from_local']
        if start_f['From'] in task_airports or flight['To'] in task_airports:
            trip_struct['id'] = start_f['Trip_id']
            trip_struct['from'] = start_f['From']
            trip_struct['to'] = flight['To']
            trips.append(trip_struct)
        start_f = flight
    else:
         pauses.append(flight['Time_from_local'] - start_f['Time_to_local'])
print(trips)



[{'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'f

In [473]:
# БЕЗ ПЕРЕСАДОК
df.loc[(df['From'] == 'DXB') & (df['To'] == 'BKK'),'flag'] = 'without_transfer'
df



,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to,flag
61,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,830.0,NaN
60,Qatar Airways,QR,N,0,None,DOH,830,OnwardPricedItinerary,2018-10-27 20:40:00,2018-10-27 20:40:00,2018-10-28 06:55:00,2018-10-28 02:55:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
77,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,NaN
76,Qatar Airways,QR,N,0,None,DOH,832,OnwardPricedItinerary,2018-10-28 08:30:00,2018-10-28 08:30:00,2018-10-28 18:45:00,2018-10-28 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
194,Qatar Airways,QR,M,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,834.0,NaN
193,Qatar Airways,QR,M,0,None,DOH,834,OnwardPricedItinerary,2018-10-28 01:20:00,2018-10-28 01:20:00,2018-10-28 11:35:00,2018-10-28 07:35:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
63,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,836.0,NaN
62,Qatar Airways,QR,N,0,None,DOH,836,OnwardPricedItinerary,2018-10-28 01:55:00,2018-10-28 01:55:00,2018-10-28 12:10:00,2018-10-28 08:10:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
103,Qatar Airways,QR,S,0,None,DWC,1033,OnwardPricedItinerary,2018-10-27 06:25:00,2018-10-27 05:25:00,2018-10-27 06:30:00,2018-10-27 06:30:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,NaN
102,Qatar Airways,QR,S,0,None,DOH,832,OnwardPricedItinerary,2018-10-27 08:30:00,2018-10-27 08:30:00,2018-10-27 18:45:00,2018-10-27 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN


In [463]:
# количество часов рейс

df['flight_time'] = df.apply(lambda row: row['Time_to_local'] - row['Time_from_local'],axis=1)

df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to,flight_time
61,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,830.0,01:05:00
60,Qatar Airways,QR,N,0,None,DOH,830,OnwardPricedItinerary,2018-10-27 20:40:00,2018-10-27 20:40:00,2018-10-28 06:55:00,2018-10-28 02:55:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
77,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,01:05:00
76,Qatar Airways,QR,N,0,None,DOH,832,OnwardPricedItinerary,2018-10-28 08:30:00,2018-10-28 08:30:00,2018-10-28 18:45:00,2018-10-28 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
194,Qatar Airways,QR,M,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,834.0,01:05:00
193,Qatar Airways,QR,M,0,None,DOH,834,OnwardPricedItinerary,2018-10-28 01:20:00,2018-10-28 01:20:00,2018-10-28 11:35:00,2018-10-28 07:35:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
63,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,836.0,01:05:00
62,Qatar Airways,QR,N,0,None,DOH,836,OnwardPricedItinerary,2018-10-28 01:55:00,2018-10-28 01:55:00,2018-10-28 12:10:00,2018-10-28 08:10:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
103,Qatar Airways,QR,S,0,None,DWC,1033,OnwardPricedItinerary,2018-10-27 06:25:00,2018-10-27 05:25:00,2018-10-27 06:30:00,2018-10-27 06:30:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,01:05:00
102,Qatar Airways,QR,S,0,None,DOH,832,OnwardPricedItinerary,2018-10-27 08:30:00,2018-10-27 08:30:00,2018-10-27 18:45:00,2018-10-27 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00


In [452]:
import io
import requests
import pandas as pd
import redis
import pytz
from tzlocal import get_localzone
from datetime import datetime

r = redis.Redis('localhost')


def search_timezone_by_iata(search_code):
    status = check_iata_spr()
    if status == 200:
        search_code = search_code.upper()
        timezone = r.hget('iata_timezone', search_code)
        if timezone is not None:
            return {'code': 200, 'data': timezone.decode('UTF-8')}
        return {'code': 500, 'msg': 'No data by key in redis'}
    return status


def get_iata_country_spr():
    url = "https://raw.githubusercontent.com/opentraveldata/opentraveldata/master/opentraveldata/optd_por_public.csv"
    try:
        s = requests.get(url).content
    except ConnectionError:
        return {'code': 500, 'msg': 'ConnectionError'}
    country_map = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='^')
    iata_dict = country_map.loc[:,
                ['iata_code', 'country_code', 'timezone']].drop_duplicates(
        subset=['iata_code']).to_dict(orient='records')
    iata_dict = {k['iata_code']: k['timezone'] for k in iata_dict}
    r.hmset('iata_timezone', iata_dict)
    check_r = r.exists('iata_timezone')
    if check_r:
        return {'code': 200}
    else:
        return {'code': 500, 'msg': 'No data in redis'}


def check_iata_spr():
    if not r.exists('iata_timezone'):
        load_spr = get_iata_country_spr()
        if 'code' in load_spr:
            if load_spr['code'] == 200:
                return 200
        return load_spr
    else:
        return 200


def get_time_by_local(iata_code, dt):
    timezone = search_timezone_by_iata(iata_code)
    if timezone['code'] == 200:
        timezone = pytz.timezone(timezone['data'])
        timezone_dt = timezone.localize(dt)
        local_zone = pytz.timezone(str(get_localzone()))
        local_dt = timezone_dt.astimezone(local_zone)
        local_dt = datetime.strftime(local_dt, '%Y-%m-%d %T')
        return {'code': 200, 'data': local_dt}
    return timezone


# t1 = datetime.now()
print(get_time_by_local('HKT', datetime(2002, 10, 27, 6, 0, 0)))
# print(datetime.now() - t1)


{'code': 200, 'data': '2002-10-27 02:00:00'}


In [608]:
a = prices.groupby(['Trip_id']).agg({'Charges':'sum'}).reset_index()

In [613]:
a.loc[a['Trip_id'] == -9164081078935422697,'Charges'].values[0]

14364.0

In [614]:
prices

,Charges,Person_type,Rate_type,Trip_id,currency
0,167.0,SingleAdult,BaseFare,5535279362652559901,SGD
1,215.7,SingleAdult,AirlineTaxes,5535279362652559901,SGD
2,382.7,SingleAdult,TotalAmount,5535279362652559901,SGD
3,129.0,SingleChild,BaseFare,5535279362652559901,SGD
4,215.7,SingleChild,AirlineTaxes,5535279362652559901,SGD
5,344.7,SingleChild,TotalAmount,5535279362652559901,SGD
6,20.0,SingleInfant,BaseFare,5535279362652559901,SGD
7,20.0,SingleInfant,TotalAmount,5535279362652559901,SGD
8,233.0,SingleAdult,BaseFare,-119555841769271300,SGD
9,152.4,SingleAdult,AirlineTaxes,-119555841769271300,SGD


In [624]:
local_zone = get_localzone()
local_zone
    

<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>

In [626]:
pytz.country_timezones(str(local_zone))

KeyError: 'EUROPE/MOSCOW'

In [627]:
for i in pytz.country_timezones:
    print(i)

AD
AE
AF
AG
AI
AL
AM
AO
AQ
AR
AS
AT
AU
AW
AX
AZ
BA
BB
BD
BE
BF
BG
BH
BI
BJ
BL
BM
BN
BO
BQ
BR
BS
BT
BW
BY
BZ
CA
CC
CD
CF
CG
CH
CI
CK
CL
CM
CN
CO
CR
CU
CV
CW
CX
CY
CZ
DE
DJ
DK
DM
DO
DZ
EC
EE
EG
EH
ER
ES
ET
FI
FJ
FK
FM
FO
FR
GA
GB
GD
GE
GF
GG
GH
GI
GL
GM
GN
GP
GQ
GR
GS
GT
GU
GW
GY
HK
HN
HR
HT
HU
ID
IE
IL
IM
IN
IO
IQ
IR
IS
IT
JE
JM
JO
JP
KE
KG
KH
KI
KM
KN
KP
KR
KW
KY
KZ
LA
LB
LC
LI
LK
LR
LS
LT
LU
LV
LY
MA
MC
MD
ME
MF
MG
MH
MK
ML
MM
MN
MO
MP
MQ
MR
MS
MT
MU
MV
MW
MX
MY
MZ
NA
NC
NE
NF
NG
NI
NL
NO
NP
NR
NU
NZ
OM
PA
PE
PF
PG
PH
PK
PL
PM
PN
PR
PS
PT
PW
PY
QA
RE
RO
RS
RU
RW
SA
SB
SC
SD
SE
SG
SH
SI
SJ
SK
SL
SM
SN
SO
SR
SS
ST
SV
SX
SY
SZ
TC
TD
TF
TG
TH
TJ
TK
TL
TM
TN
TO
TR
TT
TV
TW
TZ
UA
UG
UM
US
UY
UZ
VA
VC
VE
VG
VI
VN
VU
WF
WS
YE
YT
ZA
ZM
ZW


In [630]:
get_local

{'D': 'T',
 'E': 'Y',
 'F': 'W',
 'G': 'V',
 'I': 'V',
 'L': 'T',
 'M': 'Z',
 'O': 'T',
 'Q': 'M',
 'R': 'T',
 'S': 'W',
 'T': 'Y',
 'U': 'V',
 'W': 'Z',
 'X': 'S',
 'Z': 'U',
 'A': 'Z',
 'B': 'S',
 'H': 'T',
 'J': 'T',
 'N': 'V',
 'Y': 'U',
 'C': 'V',
 'K': 'T',
 'V': 'T',
 'P': 'N'}